# Face PCA export code

This code is for PCA analysis of face coordinates and export of that database

## Preliminaries

Getting the plotly ready, and adding the corresponding folder

In [1]:
%Getting ready for plotly
imatlab_export_fig('fig2plotly')
getplotlyoffline('http://cdn.plot.ly/plotly-latest.min.js')

%folders
folder.codePath = 'C:\Users\tzarzar\Box Sync\Research\GeneralCode\Matlab Code';
addpath(genpath(folder.codePath));
folder.dataPath16 = 'R:\ShriverLab\Facial features input files and databases\PSU_KU_WFS2016';
folder.databases  = 'C:\Users\tzarzar\Box Sync\Research\FacialSD\DataBases';
folder.results    = 'C:\Users\tzarzar\Box Sync\Research\FacialSD\Results\PCA';


Success! You can generate your first offline graph
using the 'offline' flag of fig2plotly as follows:

>> plot(1:10); fig2plotly(gcf, 'offline', true);



In [2]:
cd(folder.dataPath16); 
load PENNDATA_AccumulatedShapeData;% loading faces (Data)
load RefScan;% loading AM (RefScan)
load NewMaskIndex;% loading AM trimming index; (MaskIndex)
crop(RefScan,'VertexIndex',MaskIndex);% trimming the RefScan
cd(folder.databases);
Covariates = readtable('Covariates.csv');

In [3]:
[keep1, keep2] = GetIntersection(Data, Covariates); 
Data           = reduceData(Data,keep1); 
Covariates     = Covariates(keep2,:);
nFaces         = length(Data.Names);

In [4]:
length(Covariates.ID) 
length(Data.Names)


ans =

        5939


ans =

        5939



## Some descriptive stats

In [5]:
[statsarray,sex] = grpstats([Covariates.Age, Covariates.Height, Covariates.Weight], Covariates.Sex,{'mean','gname'})


statsarray =

   28.1771  164.0958   64.9914
   28.3316  177.5127   79.5394
   28.0000  171.1000   69.5500
   19.0000  162.3000   60.7000
   31.0000  182.0000   76.1000


sex =

  5×1 cell array

    'Female'
    'Male'
    'Male (identified female)'
    'Female (identified male)'
    'Male (assigned intersex at birth)'



## PCA Analysis and export

GPA

In [6]:
TotalShape = [Data.Shape Data.NormShape];% concatenation of original faces (Data.Shape) and their reflections (Data.NormShape);
model      = shapePCA;% creating an empty shape space object
model.RefScan = clone(RefScan);% defining the AM that was used to create the shape space
AlignedData   = LSGenProcrustes(model,TotalShape,true,3,RefScan);% run

Starting parallel pool (parpool) using the 'local' profile ...
connected to 4 workers.


DECOMPOSING FACES INTO COMPONENTS OF SYMMETRY AND ASSYMETRY

In [7]:
OrigHead = AlignedData(:,1:nFaces);
ReflHead = AlignedData(:,nFaces+1:end);
SymHead  = (OrigHead+ReflHead)/2;% facial symmetry component
AsymHead = (OrigHead-ReflHead)+mean(SymHead,2);% facial asymmetry component

Face PCA

In [8]:
getAverage(model, SymHead);% compute the average shape
getModel(model, SymHead);
means = mean(SymHead,2);

In [9]:
stripPercVar(model, 98); 
model


model = 

  shapePCA with properties:

    AvgVertices: [3×6790 double]
         AvgVec: [20370×1 double]
        RefScan: [1×1 meshObj]
        Average: [1×1 meshObj]
         EigVal: [87×1 double]
         EigVec: [20370×87 double]
         Tcoeff: [5939×87 double]
       AvgCoeff: [87×1 double]
      Centering: 1
           nrEV: 87
         EigStd: [87×1 double]
      Explained: [87×1 double]
              n: 5939
              U: [5939×87 double]
              S: [87×1 double]
              V: [20370×87 double]
           Type: 'shapePCA'



## Creating database for export

In [12]:
PCnames = strseq("PC", 1:model.nrEV);
coeffs  = [Covariates, array2table(model.Tcoeff, 'VariableNames', PCnames)]; 

In [15]:
cd(folder.results)
csvwrite("eigenvalues.csv", model.EigVal)
csvwrite("eigenvectors.csv", model.EigVec)
csvwrite("means.csv", means)
csvwrite("facets.csv", model.Average.Faces')
writetable(coeffs,'coeffs.csv')